# Описание проекта: 

Данные были представленны на основании 3 х фаилов. еобходимо проанализировать данные. Собранная статистика учитывает оценку "Покупательская активность". На основе данных нужно построить модель, которая сможет правильно прогнозировать данный признак. Произвести оценку модели. Дать рекомендации на основании произведенных исследований.

In [ ]:
# инсталяция библиотек
!pip install skimpy

!pip install optuna 
!pip install sketch
!pip install mlxtend
!pip install optuna-integration
!pip install imblearn
#!pip install yellowbrick
!pip install phik 
!pip install shap
!pip install scikit-learn
!!pip install sklearn
#df=df.rename(columns=lambda x:x.lower().replace(' ','_'))

In [ ]:
# загрузка библиотек

import pandas as pd
import numpy as np

import phik
import shap 
import sklearn.utils
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import (confusion_matrix,mean_squared_error,precision_recall_curve,roc_auc_score,
                             r2_score,accuracy_score,make_scorer,f1_score,roc_curve,RocCurveDisplay,
                             mean_absolute_error,recall_score,precision_score,ConfusionMatrixDisplay)

from sklearn.feature_selection import SelectKBest,f_classif, mutual_info_classif  
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import (StandardScaler,MinMaxScaler,LabelEncoder,RobustScaler,
                                   OneHotEncoder,PolynomialFeatures,OrdinalEncoder) 

from sklearn.model_selection import (train_test_split,GridSearchCV,RandomizedSearchCV)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,plot_tree ,export_text 
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 

from optuna import distributions,visualization 
from optuna.integration import OptunaSearchCV
from phik.report import plot_correlation_matrix
from phik import phik_matrix


pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format
RANDOM_STATE = 42
TEST_SIZE = 0.25
#pd.options.mode.copy_on_write = True

In [ ]:
# функиция преобразования в bool и запоминания значений в словарь
def bul(dat,col,tru,nt):
    dat[col][dat[col]==tru]=1 
    dat[col][dat[col]==nt]=0
    dat[col]=dat[col].astype('bool')
    #voc_k[col]=(nt,tru)

## Загрузка данных

In [ ]:
# m-main, mm- market money, mt - market time, с -cost
df_m = pd.read_csv('C:\Users\rs_mi\git_hub\teacher_learn\market_file.csv', sep =',', decimal='.')
df_mm =pd.read_csv('C:\Users\rs_mi\git_hub\teacher_learn\market_money.csv', sep =',',decimal='.')
df_mt=pd.read_csv('C:\Users\rs_mi\git_hub\teacher_learn\market_time.csv', sep =',',decimal='.')
df_c=pd.read_csv('C:\Users\rs_mi\git_hub\teacher_learn\money.csv', sep =';',decimal=',')

dat_name=[df_m,df_mm,df_mt,df_c]
for d in dat_name:
    display(d.tail(5))
    d.info()
    display (d.nunique())
    
    

## Предобработка данных

In [ ]:
#преданализ данных
for d in dat_name:
    #display(d.columns)
    for i in d.columns:
        if d[i].nunique()<15 :
            display(i,d[i].unique() )

#исправление опечатки
df_m['Тип сервиса'][df_m['Тип сервиса']=='стандартт']='стандарт'
df_mt['Период'][df_mt['Период']=='предыдцщий_месяц']='предыдущий_месяц'

In [ ]:
# преоюразуем и сократим базы по маркетигу и выручке
df_mt.info()
display(df_mt['Период'].unique() )
display(df_m['Тип сервиса'].unique() )

m1=df_mm[df_mm['Период']=='текущий_месяц']
m1=m1.drop('Период',axis=1).copy()
m1=m1.rename(columns={'Выручка': 'Выручка_ТМ'})

m2=df_mm[df_mm['Период']=='предыдущий_месяц']
m2=m2.rename(columns={'Выручка': 'Выручка_ПМ'})
m2=m2.drop('Период',axis=1).copy()

m3=df_mm[df_mm['Период']=='препредыдущий_месяц']
m3=m3.rename(columns={'Выручка': 'Выручка_ППМ'})
m3=m3.drop('Период',axis=1).copy()

df_mm_n=m1.merge(m2, on='id', how='left' ) 
df_mm_n=df_mm_n.merge(m3, on='id', how='left' ) 


m1=df_mt[df_mt['Период']=='текущий_месяц']
m1=m1.drop('Период',axis=1).copy()
m1=m1.rename(columns={'минут': 'минут_ТМ'})

m2=df_mt[df_mt['Период']=='предыдущий_месяц']
m2=m2.drop('Период',axis=1).copy()
m2=m2.rename(columns={'минут': 'минут_ПМ'})

m2.info()
df_mt_n=m1.merge(m2, on='id', how='left' ) 


df_mm_n['дельта_выр_пред'] =df_mm_n['Выручка_ПМ'] -df_mm_n['Выручка_ППМ']
df_mm_n['дельта_выр_тек'] =df_mm_n['Выручка_ТМ'] -df_mm_n['Выручка_ПМ']

df_mt_n.info()
df_mm_n.info()

In [ ]:
# преобразовываем 2-х категорийные параметры в boolewn
bul(df_m,'Покупательская активность','Снизилась','Прежний уровень')
bul(df_m,'Разрешить сообщать','да','нет')
bul(df_m,'Тип сервиса','премиум','стандарт')

df_m.info()

In [ ]:
# преобразуем длительность в днях  в месяцы
df_m['Длительность']=round(df_m['Длительность']/30.42,0).astype('int')
df_mcc=df_m.copy()
# поскольку "популярная категория" не поддается ранжированию, то преобразование в числоврй тип предлагается сделать с помощью OHE

# !!! при построении модели не забыть про Дамми-ловушку - удалить признак

#m_ohe=OneHotEncoder()
#d_ohe = m_ohe.fit_transform(df_m[['Популярная_категория']],sparse_output=False)
#encoder_col_names = m_ohe.get_feature_names_out()
#df_ohe = pd.DataFrame(d_ohe, columns=encoder_col_names )
#df_ohe=df_ohe.astype('bool')
#df_m= pd.concat([df_m,df_ohe], axis=1)
#df_m=df_m.drop(['Популярная_категория'], axis=1)

df_m.info()

### Промежуточные выводы предобработки :


- исправлена опечатка в периоде в базе с выручкой 
- исправлена опечатка в "Тип сервиса" в основной базе
- уникальные id клиентов по всем базам совпадаеют полностью, в базе активнось показана активность только за 3 месяча, в базе выручка-за 2 месяца
- преобразованны данные в числовые/bool
- преобразовали данные по времени по клиентам (выручка, маркетинг)
- "оцифровали" 'Популярная_категория' OHE - не "цифровых" данных больше нет
- преобразовали "Длительность" в месяцас округлением до целого

## Исследовательский анализ данных

In [ ]:
# анализ данных по времени 
n_k_p=df_mt_n.columns
n_k_p=n_k_p.drop('id').copy()

for n in n_k_p:
    display(n)
    plt.hist(df_mt_n[n],bins=50)
    plt.show()

In [ ]:

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

n_k_p=df_m.select_dtypes(include='float').columns.to_list()
# создадим два подграфика ax_box и ax_hist
# кроме того, укажем, что нам нужны:

for n in n_k_p:
    fig, (ax_box, ax_hist) = plt.subplots(2, # две строки в сетке подграфиков,
                                    sharex = True, # единая шкала по оси x и
                                    gridspec_kw = {'height_ratios': (.15, .85)}) # пропорция 15/85 по высоте
    
    sns.boxplot(x = df_mcc[n], ax = ax_box)
    plt.hist(df_mcc[n],bins=50)
    
    # добавим подписи к каждому из графиков через метод .set()
    ax_box.set(xlabel = '') # пустые кавычки удаляют подпись (!)
    ax_hist.set(xlabel = n)
    ax_hist.set(ylabel = 'count')
    # выведем результат
    plt.show()

    

In [ ]:
for i in df_mcc.columns:
    if df_mcc[i].nunique()<15:
        sns.countplot(y=i, data=df_mcc)
        plt.show()
        
plt.show()
##display(df_m.describe())

In [ ]:
# анализ данных по выручке 
n_k_p=df_mm_n.columns
n_k_p=n_k_p.drop('id').copy()

for n in n_k_p:
    fig, (ax_box, ax_hist) = plt.subplots(2, # две строки в сетке подграфиков,
                                    sharex = True, # единая шкала по оси x и
                                    gridspec_kw = {'height_ratios': (.20, .80)}) # пропорция 15/85 по высоте
    sns.boxplot(x = df_mm_n[n], ax = ax_box)
    plt.hist(df_mm_n[n],bins=50)
    # добавим подписи к каждому из графиков через метод .set()
    ax_box.set(xlabel = '') # пустые кавычки удаляют подпись (!)
    ax_hist.set(xlabel = n)
    ax_hist.set(ylabel = 'count')
    # выведем результат
    plt.show()       
   

In [ ]:
#исследуем "выбросы" по выручке
display('Выручка_ТМ>15000',df_mm_n[df_mm_n['Выручка_ТМ']>15000])
display('Выручка_ПМ<2500',df_mm_n[df_mm_n['Выручка_ПМ']<2500])
display('Выручка_ППМ<3500',df_mm_n[df_mm_n['Выручка_ППМ']<3500])
display('дельта_выр_тек',df_mm_n[df_mm_n['дельта_выр_тек']<-1000])

display('клиенты с нулевой выручкой', df_mm[df_mm['Выручка']==0])

#предлагается удалить единичные выбросы
df_mm_n=df_mm_n[df_mm_n['Выручка_ТМ']<15000]
df_mm_n=df_mm_n[df_mm_n['Выручка_ПМ']>2500]
                
n_k_p=['Выручка_ТМ','Выручка_ПМ','Выручка_ППМ']
for n in n_k_p:
    display(n)
    plt.hist(df_mm_n[n],bins=50)
    #sns.distplot(df_mm_n[n].to_numpy())
    plt.show()

In [ ]:
n_k_p=[ 'Маркет_актив_6_мес','Акционные_покупки','Длительность','Страниц_за_визит']

for n in n_k_p:
    fig, (ax_box, ax_hist) = plt.subplots(2, # две строки в сетке подграфиков,
                                    sharex = True, # единая шкала по оси x и
                                    gridspec_kw = {'height_ratios': (.15, .85)}) # пропорция 15/85 по высоте
    
    sns.boxplot(x = df_m[n], ax = ax_box)
    plt.hist(df_m[n],bins=50)
    
    # добавим подписи к каждому из графиков через метод .set()
    ax_box.set(xlabel = '') # пустые кавычки удаляют подпись (!)
    ax_hist.set(xlabel = n)
    ax_hist.set(ylabel = 'count')
    # выведем результат
    plt.show()    
    
    
   

dat_name=[df_m,df_mm_n,df_mt_n,df_c]
for d in dat_name:
    display(d.describe(include='all'))
#sns.pairplot(df_mcc, hue='Покупательская активность')

### Вывод : 
- клиенты с id =215359,215357,215348  не совершали покупки в предыдущие  2 месяца, клиент с id=215380 показал выручку более 100 тыс - такое реально, но для постоения модели лучше удалить. Удалим и клиентов с 0 выручкой в предыдущих	периодах. После этих преобразований графики значений стали более нормализированы
- данные по категориальным признакам не особо равномерно распределенны
- "Акционные покупки" имеют 2 "горба"
- "привлечение" пользователей на сайт происходит неравномерно, показатель снизился за последне 3 месяца

##  Объединение таблиц

In [ ]:
#t - total
df_t=pd.merge(df_mm_n,df_mt_n, on='id', how='left' )
df_t=pd.merge(df_t,df_m, on='id', how='left' )
df_t=pd.merge(df_t,df_c, on='id', how='left' )

#display(sorted(df_t['Длительность'].unique()))
display(sorted(df_t['минут_ТМ'].unique()))
display(sorted(df_t['минут_ПМ'].unique()))
display(sorted(df_t['Страниц_за_визит'].unique()))

df_t=df_t.drop('id',axis=1).copy()        
  
df_t.info()
display(df_t.head(5))
display(df_t.describe(include='all'))

### объединение прошло успешно

##  Корреляционный анализ

In [ ]:
# оценим линейную зависимость с помощью метода  spearman
display (' матрица spearman')
df_tk=df_t.drop(['Покупательская активность','Популярная_категория'],axis=1).copy()

plt.figure(figsize = (15,10))
sns.heatmap(df_tk.corr(method='spearman'), fmt='.2f', annot=True,cmap='coolwarm')

plt.show()     

display ('spearman')
correlations = df_tk.corrwith(df_t['Покупательская активность'],
                                           method='spearman').sort_values(ascending=False)
plot = sns.barplot(y=correlations.index, x=correlations)
plot.figure.set_size_inches(13, 10)


# создаём список с наименованиями непрерывных признаков
interval_cols = ['Маркет_актив_6_мес', 
                 'Выручка_ТМ','Выручка_ПМ','Выручка_ППМ',
                 'дельта_выр_пред','дельта_выр_тек','Акционные_покупки','Прибыль']

# 'PHIK матрица'
display ('PHIK матрица df_t')
phik_overview = phik_matrix(df_t, interval_cols=interval_cols) 
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(15, 10)
) 
plt.show() 

 ###  Вывод    :
- мультиколленираности не наблюдается 
- целевая переменная не зависит от следующих признкаков в обоих случаях анализа (значения менее 0.2)   : 
  - 'Разрешить сообщать' 
  - 'Маркет_актив_тек_мес'
  - 'дельта_выр_тек' 
  - 'Прибыль' 
  - 'Выручка_ТМ'
  - 'Выручка_ПМ'
  - 'Длительность'
  - 'тип сервиса'
  - 'тип сервиса'
  

##  Использование пайплайнов

- построим 4 вида моделей определения целевого параметра по выборочной базе. 
  - DecisionTree 
  - KNaighborsClassifier
  - LR
  - SVC
- в качестве оценки модели примменим ROC_AUС - метрики, учитывающей степень уверенности модели в своём прогнозе. Выбор обусловлен в т.ч. 
  - универсальностью применения для любой из вышеперечисленных моделей
  - остается неизменным при изменении соотношения между классами, что делает его удобным для несбалансированных наборов данных, которые у нас наблюдаются
  - Интуитивно понятная интерпретация: Площадь под кривой предоставляет легко интерпретируемую метрику качества модели

### подготовка общего pipeline

In [ ]:

df_full=df_t.copy()
df_full['Популярная_категория']=df_full['Популярная_категория'].astype('string').copy()
df_full=df_full.drop(['Разрешить сообщать',
                      'Маркет_актив_тек_мес',
                      'дельта_выр_тек',
                      'Прибыль',
                      'Выручка_ТМ',
                      'Ошибка_сервиса',
                      'Тип сервиса', 
                      'Длительность',
                      'Выручка_ПМ'
                     ], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    df_full.drop(['Покупательская активность'], axis=1),
    df_full['Покупательская активность'],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = df_full['Покупательская активность'])

# создаём списки с названиями признаков
ohe_columns = ['Средний_просмотр_категорий_за_визит',
               'Неоплаченные_продукты_штук_квартал',
                'Популярная_категория']

num_columns = [ 'Маркет_актив_6_мес','Акционные_покупки' ,
               'дельта_выр_пред', 'Страниц_за_визит',
               'минут_ТМ','минут_ПМ',
               'Выручка_ППМ']


# создаём пайплайн для подготовки признаков из списка ohe_columns: заполнение пропусков и OHE-кодирование
ohe_pipe = Pipeline([('ohe', OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False, dtype=np.bool_))])                 

# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('num', MinMaxScaler(), num_columns),
    ], 
    remainder='passthrough',verbose_feature_names_out=True
)

# создаём итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))])

### DecisionTreeClassifier

In [ ]:
# словарь для модели DecisionTreeClassifier()
param_grid = {'models':distributions.CategoricalDistribution([DecisionTreeClassifier(random_state=RANDOM_STATE)]),
              'models__min_samples_leaf': distributions.IntDistribution(3, 5),
              'models__max_depth': distributions.IntDistribution(3, 15),
              'models__max_features': distributions.IntDistribution(3, 26),
              'preprocessor__num':distributions.CategoricalDistribution([StandardScaler(), MinMaxScaler(), 'passthrough'])
             }   

   
# RandomizedSearchCV( param_distributions  GridSearchCV param_grid
randomized_search = OptunaSearchCV(
    pipe_final, 
    param_distributions=param_grid, 
    cv=5,
    scoring='roc_auc',
    error_score='raise',
    n_jobs=-1
)


randomized_search.fit(X_train, y_train)

print ('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели ROC_AUC:', randomized_search.best_score_)

###  KNeighborsClassifier 

In [ ]:
# словарь для модели KNeighborsClassifier() 
param_grid = {'models': distributions.CategoricalDistribution([KNeighborsClassifier(n_jobs=-1)]),
              'models__n_neighbors': distributions.IntDistribution(18, 55),
              'preprocessor__num': distributions.CategoricalDistribution([StandardScaler(), MinMaxScaler(), 'passthrough'])}   
   
# RandomizedSearchCV( param_distributions  GridSearchCV param_grid
randomized_search = OptunaSearchCV(
    pipe_final, 
    param_distributions=param_grid, 
    cv=6,
    scoring='roc_auc',
    error_score='raise',
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)

print ('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели ROC_AUC:', randomized_search.best_score_)

### LogisticRegression
 

In [ ]:
# словарь для модели LogisticRegression 'newton-cg','newton-cholesky', 'sag','lbfgs', penalty='l1',
param_grid = {'models':distributions.CategoricalDistribution([LogisticRegression(random_state=RANDOM_STATE,
                                                                                 max_iter=500,n_jobs=-1)]),
              'models__C':distributions.FloatDistribution(0.2, 3, step=0.3),
              'models__solver' :distributions.CategoricalDistribution(['newton-cg','newton-cholesky', 'sag','lbfgs',
                                                                       'liblinear', 'saga']),
              'preprocessor__num': distributions.CategoricalDistribution([StandardScaler(),
                                                                          MinMaxScaler(), 'passthrough'])}

   
# RandomizedSearchCV( param_distributions  GridSearchCV param_grid
randomized_search = OptunaSearchCV(
    pipe_final, 
    param_distributions=param_grid, 
    cv=10,
    scoring='roc_auc',
    error_score='raise',
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)
print ('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели ROC_AUC:', randomized_search.best_score_)

### SVC
               

In [ ]:
# словарь для модели SVC
   
param_grid ={'models': distributions.CategoricalDistribution([SVC(random_state=RANDOM_STATE,cache_size=8000,gamma='auto')]),
             'models__kernel':distributions.CategoricalDistribution(['linear','rbf']),
             'models__C':distributions.FloatDistribution(0.5, 3, step=0.5),
             'preprocessor__num': distributions.CategoricalDistribution([StandardScaler(), MinMaxScaler(), 'passthrough'])}
   
#@ RandomizedSearchCV( param_distributions  GridSearchCV param_grid
randomized_search = OptunaSearchCV(
    pipe_final, 
    param_distributions=param_grid, 
    cv=6,
    scoring='roc_auc',
    error_score='raise',
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)
print ('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели ROC_AUC:', randomized_search.best_score_)

### сбор лучших моделей по Randomized_search
(дрянь переборщик - разброс по значениям  при каждом запуске значителен (до сотых))
Лучшая модель и её параметры:
- DT

  Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.bool_'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Средний_просмотр_категорий_за_визит',
                                                   'Неоплаченные_продукты_штук_квартал',
                                                   'Популярная_категория']),
                                                 ('num', StandardScaler(),
                                                  ['Маркет_актив_6_мес',
                                                   'Акционные_покупки',
                                                   'дельта_выр_пред',
                                                   'Страниц_за_визит',
                                                   'минут_ТМ', 'минут_ПМ',
                                                   'Выручка_ППМ'])])),
                ('models',
                 DecisionTreeClassifier(max_depth=11, max_features=6,
                                        min_samples_leaf=3, random_state=42))])
   Метрика лучшей модели ROC_AUC: 0.8447141898096444

Лучшая модель и её параметры:
- KNeighborsClassifie

  Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.bool_'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Средний_просмотр_категорий_за_визит',
                                                   'Неоплаченные_продукты_штук_квартал',
                                                   'Популярная_категория']),
                                                 ('num', StandardScaler(),
                                                  ['Маркет_актив_6_мес',
                                                   'Акционные_покупки',
                                                   'дельта_выр_пред',
                                                   'Страниц_за_визит',
                                                   'минут_ТМ', 'минут_ПМ',
                                                   'Выручка_ППМ'])])),
                ('models', KNeighborsClassifier(n_jobs=-1, n_neighbors=20))])
    Метрика лучшей модели ROC_AUC: 0.8909760582086763
 
 Лучшая модель и её параметры:
- SVC

  Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.bool_'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Средний_просмотр_категорий_за_визит',
                                                   'Неоплаченные_продукты_штук_квартал',
                                                   'Популярная_категория']),
                                                 ('num', MinMaxScaler(),
                                                  ['Маркет_актив_6_мес',
                                                   'Акционные_покупки',
                                                   'дельта_выр_пред',
                                                   'Страниц_за_визит',
                                                   'минут_ТМ', 'минут_ПМ',
                                                   'Выручка_ППМ'])])),
                ('models',
                 SVC(C=1.5, cache_size=8000, gamma='auto', random_state=42))])
  Метрика лучшей модели ROC_AUC: 0.8937356295035329


Лучшая модель LogisticRegression и её параметры:
- Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.bool_'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Средний_просмотр_категорий_за_визит',
                                                   'Неоплаченные_продукты_штук_квартал',
                                                   'Популярная_категория']),
                                                 ('num', MinMaxScaler(),
                                                  ['Маркет_актив_6_мес',
                                                   'Акционные_покупки',
                                                   'дельта_выр_пред',
                                                   'Страниц_за_визит',
                                                   'минут_ТМ', 'минут_ПМ',
                                                   'Выручка_ППМ'])])),
                ('models',
                 LogisticRegression(C=2.8, max_iter=500, n_jobs=-1,
                                    random_state=42, solver='sag'))])
  Метрика лучшей модели ROC_AUC: 0.8960907197102193

In [ ]:
# Итоговая  модель-побкдитель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(C=2.6, penalty='l2', random_state=RANDOM_STATE,solver='sag',n_jobs=-1))])

pipe_final.fit(X_train, y_train)
#pipe_final.transform(X_train)
     

#model.fit(X_train,y_train)
y_train_pred = pipe_final.predict(X_train)
print(f'Метрика ROC-AUC: {roc_auc_score(y_train, y_train_pred)}')


y_test_pred = pipe_final.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
plt.figure(figsize=(9, 5.5))
plt.plot(fpr, tpr)
plt.title('ROC-curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()


precision_array, recall_array, thresholds = precision_recall_curve(y_test, y_test_pred)
plt.figure(figsize=(9, 5.5))
plt.plot(recall_array, precision_array)
plt.title('PR-curve')
plt.xlabel('recall')
plt.ylabel('precision')
plt.show()

##  Анализ важности признаков

In [ ]:
display ('Значимость признаков')
coefficients =pipe_final['models'].coef_[0]

X_train_tr =pipe_final['preprocessor'].fit_transform(X_train)  
features_names = pipe_final['preprocessor'].get_feature_names_out()

feature_importance = pd.DataFrame({'Feature': features_names, 'Importance': np.abs(coefficients)})
feature_importance = feature_importance.sort_values('Importance',ascending=True)
sns.set_style('white')
feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))
plt.show() 

In [ ]:
display ('before permutation')
# создаём полиномиальные признаки второго порядка
scaler = StandardScaler()
#scaler =MinMaxScaler()
selector = SelectKBest(f_classif, k=15)

X_trainp=pipe_final['preprocessor'].fit_transform(X_train)  
X_testp=pipe_final['preprocessor'].transform(X_test)
f_names = pipe_final['preprocessor'].get_feature_names_out()
X_trainp=pd.DataFrame(X_trainp,columns =f_names)
X_testp=pd.DataFrame(X_testp,columns =f_names)


poly = PolynomialFeatures(2)
X_trainp = poly.fit_transform((X_trainp))
X_testp = poly.transform((X_testp))


# сохранение таблицы X_test с полиномиальными признаками до стандартизации
X_test_df = pd.DataFrame(X_testp,columns = poly.get_feature_names_out())

X_test_df.info()
# стандартизируем признаки
X_train_scalled = pd.DataFrame(scaler.fit_transform(X_trainp),columns = poly.get_feature_names_out())
X_test_scalled =  pd.DataFrame(scaler.transform(X_testp),columns = poly.get_feature_names_out())


# обучаем SelectKBest
selector.fit(X_train_scalled ,y_train)
#X_trainp
# формируем выборки с лучшими признаками
features_names = X_train_scalled.columns[selector.get_support(indices=True)]

X_train_new = X_train_scalled[list(features_names)]
X_test_new = X_test_scalled[list(features_names)]



# модель на лучших признаках
pipe_final['models'].fit(X_train_new,y_train)

y_t_pred = pipe_final['models'].predict(X_train_new)
print(f'Метрика ROC-AUC: {roc_auc_score(y_train, y_t_pred)}')


coefficients =pipe_final['models'].coef_[0]

feature_importance = pd.DataFrame({'Feature': features_names, 'Importance': np.abs(coefficients)})
feature_importance = feature_importance.sort_values('Importance',ascending=True)
sns.set_style('white')
feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))
plt.show() 


# следующий шаг
def custom_f1(model, X, y):
    y_pred = pipe_final['models'].predict(X)
    score = f1_score(y, y_pred)
    return score

display ('after permutation')
result = permutation_importance(pipe_final['models'], X_test_new, y_test, scoring=custom_f1)



feature_importance = pd.DataFrame({'Feature': features_names, 'Importance': result['importances_mean']})
feature_importance = feature_importance.sort_values('Importance', ascending=True)
sns.set_style('white')
feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6)) 
plt.show() 

In [ ]:
scaler = StandardScaler()

- Вывод : после "перемешивания" ранжирование по значимости не изменилось, количество "значимых" уменьшилось
- наиболее значимыми для модели оказались "основные" признаки, а также полиноминальные признаки :
   - Страниц_за визит
   - минут_ПМ
   - минут_ТМ * страниц за визит 
   - минут_ТМ * Средний просмотр категорий за визит

In [ ]:
# шаг  3 анализ SHAP-значений Linear
explainer = shap.Explainer(pipe_final['models'], X_test_new)
shap_values = explainer(X_test_new)
plot.figure.set_size_inches(10, 8)
shap.plots.beeswarm(shap_values, max_display=20)
plt.show() 

- Страниц_за визит, минут_ПМ - оказывают "отрицальное влияние" (ОВ) (принятие целевого значения 0) влияние, однако их совокупный полиноминальный признак оказывет ПВ . Это объясняется свойством произведения отрицательных чисел
- время проведенное в ТМ * средний просмотр категогий  - отрицательное влияие.
- "выручка ППМ" оказывает ПВ (уменьшает признаки,  оказывающие ПВ)
- то, что квадрат минут ПМ оказывает ПВ еще раз водчеркивает ОВ основного признака
- Факторы влияющие на то, чтобы клиент не понизил "потребительскую активность":
  - Страниц_за визит
  - время в прошлом месяце 

In [ ]:

plot.figure.set_size_inches(10, 8)
display(df_t.loc[18])
shap.plots.waterfall(shap_values[18], max_display=15) 
plt.show() 

display(df_t.loc[218])
shap.plots.waterfall(shap_values[218], max_display=15) 
plt.show() 

 - Графики выше наглядно показывают "вклад" каждого признака в итоговое решение модели. Сводный график выше

### Анализ :
- наиболее значимыми для модели оказались "основные" признаки, а также полиноминальные признаки :
   - Страниц_за визит
   - минут_ПМ
   - минут_ТМ * страниц за визит 
   - минут_ТМ * Средний просмотр категорий за визит
   
- Страниц_за визит, минут_ПМ - оказывают "отрицальное влияние" (ОВ) (принятие целевого значения 0) влияние, однако их совокупный полиноминальный признак оказывет ПВ . Это объясняется свойством произведения отрицательных чисел
- время проведенное в ТМ * средний просмотр категогий  - отрицательное влияие.
- "выручка ППМ" оказывает ПВ (уменьшает признаки,  оказывающие ПВ)
- то, что квадрат минут ПМ оказывает ПВ еще раз водчеркивает ОВ основного признака
- Факторы влияющие на то, чтобы клиент не понизил "потребительскую активность":
  - Страниц_за визит
  - время в прошлом месяце 
  

### shap.plots.bar(shap_values, max_display=16) - не работает см https://github.com/shap/shap/issues/3673

plt.clf()
plt.cla()
plt.close()
#display(shap_values.values)
plt.setp(ax.get_xticklabels(), fontsize=12)
shap.plots.bar(shap_values, max_display=16)

#№№№№№               order=shap.Explanation.abs, clustering=None,
#№№               clustering_cutoff=0.5, show_data='auto', show=True)


#№ shap.plots.bar(shap_values.values, max_display=15)
#,x='Feature', y='Importance',)
#, max_display=15) 

plt.show() 



## Сегментация покупателей

### сегментация по признаку "количество страниц" : 

- поизведем следующую сегментацию :
  - выделим клиентов с прибылостью от 4
  - внутри группы поделим их по признаку Активность (снизилась/осталась)
   

In [ ]:
plt.hist(df_t['Прибыль'],bins=50)
plt.show()

#df_tp.info()
df_seg=df_t[df_t['Прибыль']>=4].copy()
df_seg.info()
df_seg1=df_seg[df_seg['Покупательская активность']>0]
df_seg2=df_seg[df_seg['Покупательская активность']==0]
df_seg1=df_seg1.drop(['Прибыль','Покупательская активность'],axis=1).copy()
df_seg2=df_seg2.drop(['Прибыль','Покупательская активность'],axis=1).copy()
df_seg1.info()
df_seg2.info()
display(df_seg1.describe(include='all'))
display(df_seg2.describe(include='all'))

### анализ сегментов

In [ ]:

for i in df_seg1.columns:
    if df_seg1[i].nunique()<15:
        display ('Сегмент снизилась ' )
        fig, ax = plt.subplots()
        labels = df_seg1[i].unique()
        sizes = df_seg1[i].value_counts()
        ax.pie(sizes, labels=labels, autopct='%1.2f%%')
        ax.pie(sizes, labels=labels)
        ax.set_title(i)
        plt.show()
        display ('Сегмент осталась' )
        fig, ax = plt.subplots()
        labels = df_seg2[i].unique()
        sizes = df_seg2[i].value_counts()
        ax.pie(sizes, labels=labels, autopct='%1.2f%%')
        ax.pie(sizes, labels=labels)
        ax.set_title(i)
        plt.show()

### выводы анализа
- согласно анализа по сегментам можно сделать следующие выводы:
  - следующие признаки не особо влияют на "прибыльных клиентов:
     - маркетинговая активноть текущего месяца
     - средний просмотр категорий
     - неоплаченные продукты
     - ошибка сервиса
     - популяные категории : тут явные лидеры у "понизилась" : товары для детей, косметика и посуда, против техники для красоты и бытовой техники . Наверное, недостаточный ассортимент в данных категориях, раз покупатели остались не удоволетрены (в случае определяния популярной, как самой просматриваемой). Кухонная посуда -ок. Ну и косвеный вывод, котрый можно сдлать, что "прибыльные" - это женщины, согласно набору популяных признаков
  - следующие признаки особо влияют на "прибыльных" клиентов:
      - тип сервиса
      - разрешить сообщать
   - из это можно сделать сделующие выводы: "прибыльным" не нравится , когда им "сообщают" и либо условия и ожидания от смены типа сервиса не оправдались
     

Согласен. Это один из инстереснейших разделов проекта, имющий явно прикланое и реальное применение. Этот раздел "возвращает" из мира цифр в мир реальности :-) . Согласен, что анализ никакой, вместо "экспериментов" с моделями и различными видами переборам надо бвло уделить БОЛЬШЕЕ внимание этому разделу - он более творческий..

## Выводы

`Данные были представленны на основании 3 х фаилов. В процессе подготовки данных были произведены следующие преобразования:`

- исправлена опечатка в периоде в базе с выручкой
- иправлена опечатка в "Тип сервиса" в основной базе
- уникальные id клиентов по всем базам совпадаеют полностью, в базе активнось показана активность только за 3 месяча, в базе выручка-за 2 месяца
- преобразованны данные в числовые/bool
- преобразовали данные по времени по клиентам (выручка, маркетинг)
- "оцифровали" 'Популярная_категория' OHE
- преобразовали "Длительность" в месяцас округлением до целого

`В ходе предварительного анализа было выявлено следующее:`

 - клиенты с id =215359,215357,215348 не совершали покупки в предыдущие 2 месяца, клиент с id=215380 показал выручку более 100 тыс - такое реально, постоения модели данные удалены.
- данные по категориальным признакам не особо равномерно распределенны
- "Акционные покупки" имеют 2 "горба"
- "привлечение" ползователей на сайт происходит неравномерно, показатель снизился за последне 3 месяца


`Был произведен корреляционный анализ данных. В ходе анализа выяснилось следующее:`

- мультиколленираности не наблюдается
- целевая переменная 'Покупательская активность' (1- True "уменьшилась" 0 - False "Осталось на прежнем уровне") не зависит от следующих признкаков в обоих случаях анализа (значения менее 0.1) :
  - 'Разрешить сообщать'
  - 'Маркет_актив_тек_мес'
  - 'дельта_выр_тек'
  - 'Прибыль'
  - 'Выручка_ТМ'
  - 'Популярная_категория_Домашний текстиль' 

`Были построены модели обучения (анализа) с помощью 4 методов :`
- DecisionTreeClassifier
- KNeighborsClassifier
- SVC
- LogisticRegression

C  помощью метода randomized_search с перебором по моделям и параметрам моделей был выявлен лучшый метод

`- SVC(C=2, degree=1, random_state=42, kernel='linear')`

`Проанализировали значимость переменных в модели регресии с помощью метода #`SHAP`# . Получили ледующие выводы:`
- наиболее значимыми для модели оказались "основные" признаки, а также полиноминальные признаки :
   - Страниц_за визит
   - минут_ПМ
   - минут_ТМ * страниц за визит 
   - минут_ТМ * Средний просмотр категорий за визит
   
- Страниц_за визит, минут_ПМ - оказывают "отрицальное влияние" (ОВ) (принятие целевого значения 0) влияние, однако их совокупный полиноминальный признак оказывет ПВ . Это объясняется свойством произведения отрицательных чисел
- время проведенное в ТМ * средний просмотр категогий  - отрицательное влияие.
- "выручка ППМ" оказывает ПВ (уменьшает признаки,  оказывающие ПВ)
- то, что квадрат минут ПМ оказывает ПВ еще раз водчеркивает ОВ основного признака
- Факторы влияющие на то, чтобы клиент не понизил "потребительскую активность":
  - Страниц_за визит
  - время в прошлом месяце 
  
  
Былв поизведа следующая сегментация :
  - выделим клиентов с прибыльностью от 4
  - внутри группы поделим их по признаку Активность (снизилась/осталась)  
  
 выводы анализа
- согласно анализа по сегментам можно сделать следующие выводы:
  - следующие признаки не особо влияют на "прибыльных клиентов:
     - маркетинговая активноть текущего месяца
     - средний просмотр категорий
     - неоплаченные продукты
     - ошибка сервиса
     - популяные категории : тут явные лидеры у "понизилась" : товары для детей, косметика и посуда, против техники для красоты и бытовой техники . Наверное, недостаточный ассортимент в данных категориях, раз покупатели остались не удоволетрены (в случае определяния популярной, как самой просматриваемой). Кухонная посуда -ок. Ну и косвеный вывод, котрый можно сдлать, что "прибыльные" - это женщины, согласно набору популяных признаков
  - следующие признаки особо влияют на "прибыльных" клиентов:
      - тип сервиса
      - разрешить сообщать
   - из это можно сделать сделующие выводы: "прибыльным" не нравится , когда им "сообщают" и либо условия и ожидания от смены типа сервиса не оправдались
     